In [10]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
from tensorflow.keras.layers import SpatialDropout2D, GaussianNoise, Dropout

In [2]:
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"
local_zip='/tmp/cats-and-dogs.zip'
zip_ref=zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2022-05-05 09:31:30--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.86.216.112, 2600:1417:8000:983::e59, 2600:1417:8000:9a2::e59, ...
Connecting to download.microsoft.com (download.microsoft.com)|184.86.216.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  11.6MB/s    in 66s     

2022-05-05 09:32:36 (12.0 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



In [3]:
print(len(os.listdir('/tmp/PetImages/Cat/')))
print(len(os.listdir('/tmp/PetImages/Dog/')))

12501
12501


In [4]:
try:
  os.mkdir('/tmp/cats-v-dogs')
  os.mkdir('/tmp/cats-v-dogs/training')
  os.mkdir('/tmp/cats-v-dogs/testing')
  os.mkdir('/tmp/cats-v-dogs/training/cats')
  os.mkdir('/tmp/cats-v-dogs/training/dogs')
  os.mkdir('/tmp/cats-v-dogs/testing/cats')
  os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [5]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files=[]
  for filename in os.listdir(SOURCE):
    file=SOURCE+filename
    if os.path.getsize(file)>0:
      files.append(filename)
    else:
       print(filename+'is zero length, so ignoring.')

  training_length=int(len(files)* SPLIT_SIZE)
  testing_length=int(len(files)-training_length)
  shuffled_set=random.sample(files, len(files))
  training_set=shuffled_set[0:training_length]
  testing_set=shuffled_set[-testing_length:]

  for filename in training_set:
      this_file=SOURCE+filename
      destination=TRAINING+filename
      copyfile(this_file, destination)

  for filename in testing_set:
      this_file=SOURCE +filename
      destination=TESTING+filename
      copyfile(this_file, destination)

CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size=.9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpgis zero length, so ignoring.
11702.jpgis zero length, so ignoring.


In [6]:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))

11250
11250
1250
1250


In [11]:
TRAINING_DIR="/tmp/cats-v-dogs/training/"
train_datagen=ImageDataGenerator(rescale=1.0/255.)
train_generator=train_datagen.flow_from_directory(TRAINING_DIR,
                                                  batch_size=100,
                                                  class_mode='binary',
                                                  target_size=(150,150))

VALIDATION_DIR="/tmp/cats-v-dogs/testing/"
validation_datagen=ImageDataGenerator(rescale=1.0/255.)
validation_generator=validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                            batch_size=100,
                                                            class_mode='binary',
                                                            target_size=(150,150))

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [20]:
model=tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.SpatialDropout2D(0.3),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.3),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

callback=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history=model.fit(train_generator,
                  epochs=20,
                  verbose=1,
                  validation_data=validation_generator, callbacks=[callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/20
102/225 [============>.................] - ETA: 42s - loss: 1.0995 - accuracy: 0.5083

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

225/225 [==============================] - 89s 389ms/step - loss: 0.8650 - accuracy: 0.5469 - val_loss: 0.6479 - val_accuracy: 0.6036
Epoch 2/20
225/225 [==============================] - 86s 380ms/step - loss: 0.5940 - accuracy: 0.6818 - val_loss: 0.6755 - val_accuracy: 0.6320
Epoch 3/20
225/225 [==============================] - 87s 387ms/step - loss: 0.5444 - accuracy: 0.7248 - val_loss: 0.5965 - val_accuracy: 0.6820
Epoch 4/20
225/225 [==============================] - 88s 389ms/step - loss: 0.5163 - accuracy: 0.7438 - val_loss: 0.5722 - val_accuracy: 0.6948
Epoch 5/20
225/225 [==============================] - 86s 382ms/step - loss: 0.4887 - accuracy: 0.7632 - val_loss: 0.4908 - val_accuracy: 0.7564
Epoch 6/20
225/225 [==============================] - 87s 387ms/step - loss: 0.4669 - accuracy: 0.7776 - val_loss: 0.6337 - val_accuracy: 0.6720
Epoch 7/20
225/225 [==============================] - 87s 387ms/step - loss: 0.4327 - accuracy: 0.7970 - val_loss: 0.4559 - val_accuracy: 0.7

In [23]:
model.save('Cats&Dogs.h5')